In [1]:
import pandas as pd 
import numpy as np 
import math
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv(r'../clean_data.csv')
merton = pd.read_csv(r'../merton_model_output.csv')

spy_csv = pd.read_csv(r'../hypothesis-2/SPY.csv')
spy_csv['log_return'] = np.log(spy_csv['PRC'] / spy_csv['PRC'].shift(1))
spy_csv = spy_csv.set_index('date')
spy_csv.index = pd.to_datetime(spy_csv.index)

spy_logreturns = spy_csv['log_return'].dropna()


plot = False

beta_set = []
iv_set = []
pd_set = []

tics = data['tic'].unique()
tics = np.delete(tics, 18) ## BRK.B messes everything up



for i in tics:

    stock = data[data['tic'] == str(i)]
    stock = stock.set_index('date')
    stock.index = pd.to_datetime(stock.index)

    probdef = merton[merton['tic'] == str(i)]
    probdef = probdef.set_index('date')
    probdef.index = pd.to_datetime(probdef.index)
    #print(probdef.index.duplicated().sum())


    stock_logreturns = stock['log_return']


    spy_logreturns, stock_logreturns = spy_logreturns.align(stock_logreturns, join = 'inner', axis = 0)
    
         
    X = sm.add_constant(spy_logreturns)
    y = stock_logreturns
            
    model = sm.OLS(y, X)
    results = model.fit()
         
    _, beta = results.params
    residuals = results.resid
    idio_vol = np.std(residuals)
        
    beta_set.append(beta)
    iv_set.append(idio_vol)


    
    probdefff = probdef['merton_pd'].reindex(stock_logreturns.index)

    pdd = probdefff.mean()
    pd_set.append(pdd)

    if plot == True:
        fig, ax = plt.subplots(2,2, figsize = (16,10))

        ax[1,0].plot(rolling_results['Beta'])
        ax[1,0].set_title("Rolling Beta over Time")
        ax[1,1].plot(rolling_results['Idiosyncratic Volatility'])
        ax[1,1].set_title("Rolling Idio Vol over Time")
        ax[0,0].plot(stock_filter['PRC'])
        ax[0,0].set_title('Adjusted Price Plot')
        ax[0,1].plot(rolling_pd_filt)
        fig.suptitle(f'{i} Rolling Beta and Idiosyncratic Volatility')
        plt.show()


    #########################################################################################

rolling_results = pd.DataFrame({
        'Beta': beta_set,
        'Idiosyncratic Volatility': iv_set
    })   


X2 = sm.add_constant(rolling_results)
model2 = sm.OLS(pd_set, X2)
results2 = model2.fit()

print(results2.summary())

a, b, iv = results2.params
print(f"Beta: {b*np.std(beta_set)}")
print(f"IV: {iv*np.std(iv_set)}" )

"""fig, ax = plt.subplots(2,2, figsize = (16,10))

ax[1,0].plot(rolling_results['Beta'])
ax[1,0].set_title("Systemic Risk")
ax[1,1].plot(rolling_results['Idiosyncratic Volatility'])
ax[1,1].set_title("Idiosyncratic Risk")
ax[0,0].plot(pd_set)
ax[0,0].set_title('Mean Default Probabilities')
#ax[0,1].plot(rolling_pd_filt)
fig.suptitle(f'{i} Rolling Beta and Idiosyncratic Volatility')"""


############################################################################################


fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111,projection='3d')

ax.scatter(beta_set, iv_set, pd_set, color = 'b', s = 50)

x_range = np.linspace(min(beta_set), max(beta_set), 100)
y_range = np.linspace(min(iv_set), max(iv_set), 100)
X_grid, Y_grid = np.meshgrid(x_range, y_range)
Z_grid = a + b * X_grid + iv * Y_grid

# Plot the surface of the regression plane
ax.plot_surface(X_grid, Y_grid, Z_grid, color='r', alpha=0.5, rstride=100, cstride=100)

# Labels and title
ax.set_xlabel('Systemic Risk (Beta)')
ax.set_ylabel('Idiosyncratic Volatility')
ax.set_zlabel('Probability of Default (PD)')
ax.set_title('3D Plot of Systemic Risk and Idiosyncratic Risk vs PD')

ax.view_init(elev=15, azim=90+60)
plt.show()
######################################################################################

plt.figure(figsize=(10, 6))

scatter = plt.scatter(beta_set, iv_set, c=pd_set, cmap='viridis', s=100, alpha=0.7)

cbar = plt.colorbar(scatter)
cbar.set_label('Probability of Default (PD)', rotation=270, labelpad=15)

# Labels and title
plt.xlabel('Systemic Risk (Beta)')
plt.ylabel('Idiosyncratic Volatility')
plt.title('Systemic Risk vs Idiosyncratic Volatility Colored by PD')



plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '../clean_data.csv'